## C/C++ integration with Covalent

### Interfacing with C

We use a simple example of approximating the value of $\pi$ by computing the area under one quadrant of a unit circle. To do this, we use the `Reimann` sum approach where we break the region down into many rectangles of width $dx$ and height given by the equation of the circle

$$y = \sqrt{1 - x^2}, x \in [0, 1]$$

We write a C python module, compile it into a shared library and import it in our workflow. Doing this, as far as Covalent is concerned it is dealing with an ordinary Python module with routine Python functions accepting the necessary arguments.

The module source code can be found in `cpiapprox/main.c` and the method of interest is `compute_pi`. The module follows the Python/C API standard and defines a module named `cpiapprox` with a single method `compute_pi` that accepts an `unsigned int` as an argument a.k.a the number of partitions to divide the area in. Greater the number of partition, better the approximation of $\pi$.


### Compilation

To compile the source code into a shared library that can be imported simple run

```bash
python setup.py build
python setup.py install
```
from the directory containing the source code.


In [ ]:
import cpiapprox
import covalent as ct
from typing import List

In [ ]:
@ct.electron
def compute_pi(N: int):
    """
    :param N: number of partitions to use to approximate PI
    """
    return cpiapprox.compute_pi(N)

@ct.lattice
def approximate_pi(partitions: List[int]):
    results = []
    for n in partitions:
        results.append(compute_pi(n))
    return results


dispatch_id = ct.dispatch(approximate_pi)([10, 10000, 1000000, 1000000000])
result = ct.get_result(dispatch_id, wait=True)
print(result)


### Interfacing with C++

Covalent can be interfaced with C++ in a similar manner if one can expose their `C++` library as a Python module to Covalent. There are numerous open source wrappers available like `PyBind11/SWIG`. We here show how pure C++ functions can be invoked as Covalent electrons after being exposed via `PyBind11`.

More information on `PyBind11` can be found [here](https://github.com/pybind/pybind11). To install `pybind11`, simply run

```bash
pip install pybind11
```

As an example, we write a `C++` module `cpparithmetic` that performs simple elementwise vector arthimetic on two input `std::vectors`. The source code for this module can be found in `cpparthimetic/main.cc`.

To compile the source, use the provided `setup.py` file.

```bash
python setup.py build
python setup.py install
```

In [1]:
import cpparthimetic
import covalent as ct
import random
from typing import List
import time

### C++ workflows

In [2]:
@ct.electron
def generate_lists(size: int):
    return [random.uniform(1, 10) for i in range(size)]

@ct.electron
def cppadd(a, b):
    return cpparthimetic.vecadd(a, b)

@ct.electron
def cppmul(a, b):
    return cpparthimetic.vecmul(a, b)

@ct.electron
def cppdiv(a, b):
    return cpparthimetic.vecdiv(a, b)

@ct.lattice
def cppadd_workflow(size):
    a = generate_lists(size)
    b = generate_lists(size)
    c = cppadd(a, b)
    return c

@ct.lattice
def cppmul_workflow(size):
    a = generate_lists(size)
    b = generate_lists(size)
    c = cppmul(a, b)
    return c

@ct.lattice
def cppdiv_workflow(size):
    a = generate_lists(size)
    b = generate_lists(size)
    c = cppdiv(a, b)
    return c

In [3]:
N = 2**4

In [4]:
dispatch_ids = [ct.dispatch(cppadd_workflow)(N), ct.dispatch(cppmul_workflow)(N), ct.dispatch(cppdiv_workflow)(N)]

In [5]:
results = [ct.get_result(dispatch_id, wait=True).result for dispatch_id in dispatch_ids]

In [6]:
print(results)

[[11.177078697934395, 16.3214734885964, 8.834955323755924, 8.07036795823727, 11.330697170400924, 15.65185481221301, 16.58107154170786, 11.823631680553053, 6.78491002426072, 9.826080452983105, 8.166566338084287, 11.345197831006566, 14.219363737251209, 12.49897999347533, 7.100003433959273, 11.15488621668193], [8.391131488520463, 9.14633497275894, 20.630944026679234, 50.760671585268796, 36.298117726372304, 22.301547183133145, 68.3634746550888, 13.03946228022676, 38.59259685129559, 37.58011214332306, 21.512997242192057, 60.93803853002853, 21.710472563567834, 5.539328975142298, 45.978288643374675, 35.14193071250006], [1.2462214408816936, 0.42462435746921023, 4.412856744027672, 0.3933190254969646, 0.5968472346032822, 1.0387311819518477, 0.9992225138631028, 2.4459486864861546, 1.0212944553754442, 0.4934174678610028, 1.4509336189606175, 4.8995399594386, 1.1365109969616864, 1.4189213108623573, 0.4704953274312155, 2.0912983895128163]]
